# Tarea 5

_Tarea 5_ de _Benjamín Rivera_ para el curso de __Métodos Numéricos__ impartido por _Joaquín Peña Acevedo_. Fecha limite de entrega __4 de Octubre de 2020__.

### Como ejecutar

##### Requerimientos

Este programa se ejecuto en mi computadora con la version de __Python 3.8.2__ y con estos
[requerimientos](https://github.com/BenchHPZ/UG-Compu/blob/master/MN/requerimientos.txt)

#### Jupyter

En caso de tener acceso a un _servidor jupyter_ ,con los requerimientos antes mencionados, unicamente basta con ejecutar todas las celdas de este _notebook_. Probablemente no todas las celdas de _markdown_ produzcan el mismo resultado por las 
[_Nbextensions_](jupyter-contrib-nbextensions.readthedocs.io).

#### Consola

Habrá archivos e instrucciones para poder ejecutar cada uno de los ejercicios desde la consola.

#### Si todo sale mal

<a href="https://colab.research.google.com/gist/BenchHPZ/">
    <img src="../../../assets/colab-badge.svg" 
         alt="Open In Colab"/>
</a>

En caso de que todo salga mal, tratare de dejar una copia disponible en __GoogleColab__ que se pueda ejecutar con la versión de __Python__ de _GoogleColab_

## Ejercicio 1

Considere la matriz
\begin{equation*}
    A = \begin{pmatrix}
            a^2 & a & a/2 & 1 \\
            a & -9 & 1 & 0 \\
            a/2 & 1 & 10 & 0 \\
            1 & 0 & 0 & a
        \end{pmatrix}
\end{equation*}

Da un rango de valores para $a$ de manera que garantice la convergencia del método de Jacobi.

### Resp

Por las notas del curso (ppt clase 9, diapositiva 8-41), sabemos que el método de Jacobi converge cuando la matriz $A$ es \textbf{estrictamente diagonal dominante}. Y esto es cierto cuando

\begin{equation*}
    \forall i \in [1,\dots,n] , |a_{i,i}| > \sum_{j=1, j\neq i}^{n} |a_{i,j}|
\end{equation*}

si extendemos estas desigualdades para la matriz $A$ nos queda que

\begin{eqnarray}
    sol &=&
    \begin{cases}
        |a^2| &>& |a| + |a/2| + |1| \\
        |-9|  &>& |a| + |1| + |0| \\
        |10|  &>& |a/2| + |1| + |0| \\
        |a|  &>& |1| + |0| + |0| 
    \end{cases} \\
    && \text{despues de simplificar queda que} \\
    &=& \begin{cases}
        a^2 &>& |a| + |a/2| + 1 \\
        8   &>& |a| \\
        9  &>& |a/2| \\
        |a| &>& 1 
    \end{cases} \\
    && \\
    &=& \begin{cases}
        a^2 &>& |a| + |a/2| + 1 \\
        64   &>& a^2 \\
        4*91  &>& a^2 \\
        a^2 &>& 1 
    \end{cases} \label{eq: red}\\
    && \text{realcionamos \ref{eq: red}.4 con \ref{eq: red}.3 y \ref{eq: red}.2} \\
    &=& \begin{cases}
        a^2 &>& |a| + |a/2| + 1 \\
        8^2   &>& a^2 >  1\\
        4*9^2  &>& a^2 > 1\\
    \end{cases} \label{eq: cuadrado}\\
    && \text{de esto solo nos importa \ref{eq: cuadrado}.1 y \ref{eq: cuadrado}.2} \\
    &=& \begin{cases}
        a^2 &>& 3|a|/2 + 1 \\
        8   &>& a >  1\\
    \end{cases} \label{eq: final}\\
\end{eqnarray}

Podemos calcular los intervalos de soluci\'on de \ref{eq: final}. Estos quedan

\begin{equation}
    \begin{cases}
        a^2 > 3|a|/2 + 1 &\Rightarrow& (-\infty, -2) \cup(2, \infty)  \\
        8   > a >  1     &\Rightarrow& (1, 8)
    \end{cases}
    \label{eq: casi sol}
\end{equation}

Y la solucion que buscamos es la interseccion de \ref{eq: casi sol}. De manera que, para que la matriz $A$ converja con el metodo de Jacobi, se necesita que $x \in (2,8)$.

## Ejercicio 2

Sea $A \in \mathbb R^{n\times n}$ una matriz tridiagonal y que las tres diagonales de interes se almacenan en un arreglo $B_{n \times 3}$.

Escribe las expresiones para calcular las actualizaciones de las componentes del vector $x^{i+1} = \left( x^{i+1}_0, \dots, x^{i+1}_{n-1} \right)$ de acuerdo con \textit{Gauss-Seidel}. Especificamente escribir la expresion para actualizar $x^{i+1}_0, x^{i+1}_i$ para $i = 1,2,\dots, n-2$; ademas de $x^{i+1}_{n-1}$ usando los coeficientes $a_{i,j}$ de $A$ y $b_{ij}$ de $B$. 

### Respuesta

\par Sea $A$ una matriz tridiagonal con elementos $a_{i,j}$, $B$ el arreglo descrito anteriormente con elementos $b'_{i,j}$, $b$ el vector de terminos independientes con elementos $b_i$ y $x$ el vector solucion con $x_i^{(t)}$ su elemento $i$ de la iteraci\'on $t$.

\par Se da que en el m\'etodo de \textit{Gauss-Seidel} original tenemos que los componentes $x^{(t+1)}$ se calculan siguiendo \verb|forwardSubstitution|

\begin{equation}
	x_{i}^{(t+1)} = \frac{1}{a_{i,i}}\left( b_i - \sum_{j=0}^{i-1} a_{j,j}x_{j}^{(t+1)} - \sum_{j=i+1}^{n-1} a_{i,j}x_j^{(t)}\right)
	\label{eq: GS original}
\end{equation}

\noindent pero como en este ejercicio estamos trabajando con una matriz tridiagonal, lo que implica que solo habr\'a elementos distitnos de cero en las tres diagonales de interes; entonces podemos reescribir la ecuaci\'on~\ref{eq: GS original}, lo que queda como

\begin{eqnarray*}
	x_{i}^{(t+1)} &=& \frac{1}{a_{i,i}}\left( b_i - a_{i, j-1}x_j^{t+1} - a_{i, j+1}x_j^{t}\right)
	\label{eq: GS tridiagonal} \\
	 &=& \frac{1}{b'_{1,i}}\left( b_i - b'_{i,0}x_j^{t+1} - b'_{i,2}x_j^{t}\right)
	\qquad\text{Usando el arreglo B} \label{eq: GS tridiagonal con B}
\end{eqnarray*}

\noindent esto se puede usar $\forall i = 0,1,\dots,n-1,n$ sobre el arreglo $B$.

\par Espec\'ificamente podemos definir al elemento $x_0^{t+1}$ como
    \begin{eqnarray*}
		x_0^{t+1} &=& \frac{1}{a_{0,0}} \left( b_0 - a_{0, 1}  \right) \\
			&=& \frac{1}{b_{0,1}} \left( b_0 - b'_{0, 2}  \right)
	\end{eqnarray*}
	
\noindent y para el elemento $x_{n-1}^{i+1}$ queda que
	\begin{eqnarray*}
		x_{n-1}^{i+1} &=& \frac{1}{a_{n-1,n-1}} \left( b_{n-1} - a_{n-1, n-2}  \right) \\
			&=& \frac{1}{a_{n-1,1}} \left( b_{n-1} - b'_{n-1, 0}  \right)
	\end{eqnarray*}

## Ejercicio 3

Programa el metodo de \textit{Gauss-Seidel} para resolver sistemas tridiagonales. 